# Algoritmos de optimización - Reto 3

Nombre: Álvaro Caño Soto<br>
Github: https://github.com/alvarocanosoto/Algoritmos-de-Optimizacion/tree/main/Retos <br>

In [1]:
!pip install requests
import urllib.request

!pip install tsplib95
import tsplib95

In [2]:
import random   #Libreria para generar numeros y listas aleatorias
import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...

In [7]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp"
# urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')

#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia
problem.get_weight(1, 2)

#Matriz de distancias
[ [problem.get_weight(i, j)   for i in  list(problem.get_nodes()) ]  for j in  list(problem.get_nodes()) ]

[[0,
  15,
  30,
  23,
  32,
  55,
  33,
  37,
  92,
  114,
  92,
  110,
  96,
  90,
  74,
  76,
  82,
  67,
  72,
  78,
  82,
  159,
  122,
  131,
  206,
  112,
  57,
  28,
  43,
  70,
  65,
  66,
  37,
  103,
  84,
  125,
  129,
  72,
  126,
  141,
  183,
  124],
 [15,
  0,
  34,
  23,
  27,
  40,
  19,
  32,
  93,
  117,
  88,
  100,
  87,
  75,
  63,
  67,
  71,
  69,
  62,
  63,
  96,
  164,
  132,
  131,
  212,
  106,
  44,
  33,
  51,
  77,
  75,
  72,
  52,
  118,
  99,
  132,
  132,
  67,
  139,
  148,
  186,
  122],
 [30,
  34,
  0,
  11,
  18,
  57,
  36,
  65,
  62,
  84,
  64,
  89,
  76,
  93,
  95,
  100,
  104,
  98,
  57,
  88,
  99,
  130,
  100,
  101,
  179,
  86,
  51,
  4,
  18,
  43,
  45,
  95,
  45,
  115,
  93,
  152,
  159,
  100,
  112,
  114,
  153,
  94],
 [23,
  23,
  11,
  0,
  11,
  48,
  26,
  54,
  70,
  94,
  69,
  89,
  75,
  84,
  84,
  89,
  92,
  89,
  54,
  78,
  99,
  141,
  111,
  109,
  190,
  89,
  44,
  11,
  29,
  54,
  56,
  89,
  47,
  1

In [8]:
#Funciones de la Actividad Guiada 3
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [9]:
distancia_total( [0, 32, 34, 33, 20, 35, 36, 31, 17, 7, 1, 6, 4, 3, 27, 2, 28, 29, 30, 38, 22, 39, 21, 24, 40, 23, 41, 9, 8, 10, 25, 11, 12, 18, 26, 5, 13, 19, 14, 16, 15, 37] , problem)


1326

In [20]:
#Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos,N):
  return [crear_solucion(Nodos) for _ in range(N) ]


#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
    mejor_solucion = []
    mejor_distancia = float('inf')
    for p in poblacion:
        distancia_referencia = distancia_total(p, problem)
        if distancia_referencia < mejor_distancia:
            mejor_solucion = p
            mejor_distancia = distancia_referencia
    return mejor_solucion, mejor_distancia

#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion,mutacion, problem):
    poblacion_final = copy.deepcopy(poblacion)
    while len(poblacion) > 1:
        padre1, padre2 = random.sample(poblacion, 2)
        poblacion.remove(padre1)
        poblacion.remove(padre2)
        poblacion_final.extend(Descendencia([padre1, padre2], problem, mutacion))
    return poblacion_final

#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
def Descendencia(padres, problem,mutacion):
  #Se elige un punto de corte aleatorio:
  pc = random.sample( range(1,len(padres[0])),1)[0]
  hijo1 =  Factibilizar(padres[0][:pc] + padres[1][pc:] ,problem)
  hijo2 =  Factibilizar(padres[1][:pc] + padres[0][pc:] ,problem)
  return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion,problem):
  Nodos = list(problem.get_nodes())
  nodos_desaparecidos = list(set(Nodos) - set(solucion))
  #Recorremos todos los nodos, cuando haya uno que ya esté en la lista los cambiamos por uno de la lista de nodos_desaparecidos
  for i in range(len(solucion) ):
    if solucion[i] in solucion[:i]:
      #print("\tSe repite el",solucion[i] )
      #print("\tSe cambia en la posicion ",i , " " ,solucion[i] , " por ", nodos_desaparecidos[0] )
      solucion[i] = nodos_desaparecidos.pop(0) #Cambiamos el nodo y a la vez eliminamos el nodo usado de nodos_desaparecidos
  return solucion

#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion,mutacion):
    if random.random() < mutacion:
        sel1, sel2 = random.sample(range(1, len(solucion)), 2)
        solucion[sel1], solucion[sel2] = solucion[sel2], solucion[sel1]
    return solucion
        
#Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem,poblacion, N, elitismo):
    poblacion_ordenada = sorted(
        [[distancia_total(solucion, problem), solucion] for solucion in poblacion],
        key=lambda x: x[0]
    )
    elite = [x[1] for x in poblacion_ordenada][:int(N * elitismo)]
    resto = random.sample(
        [x[1] for x in poblacion_ordenada][int(N * elitismo):],
        int(N * (1 - elitismo))
    )
    return elite + resto


In [22]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
    Nodos = list(problem.get_nodes())
    poblacion = generar_poblacion(Nodos, N)
    
    mejor_solucion, mejor_distancia = Evaluar_Poblacion(poblacion, problem)
    
    mejores_distancias = [mejor_distancia]
    
    for n in range(generaciones):
        poblacion = Cruzar(poblacion, mutacion, problem)
        poblacion = Seleccionar(problem, poblacion, N, elitismo)
        mejor_solucion, mejor_distancia = Evaluar_Poblacion(poblacion, problem)
    
        mejores_distancias.append(mejor_distancia)
        print(f"Generación {n+1}: Mejor distancia = {mejor_distancia}")
    
    return mejor_solucion, mejor_distancia

#1º intento  :2113
#sol = algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=200)

#2º intento. Aumentamos la poblacion:1654
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=200)

#3º intento. Aumentamos las generaciones:2055
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=250)

#4º intento. Aumentamos el elitismo: 1728
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.40,generaciones=250)

#5º intento. Aumentamos la mutacion : 1629 [0, 29, 9, 21, 40, 24, 39, 22, 38, 30, 28, 4, 18, 26, 5, 6, 32, 34, 33, 20, 35, 36, 31, 17, 37, 7, 8, 23, 41, 10, 25, 11, 12, 13, 19, 14, 16, 15, 1, 3, 2, 27]
sol, distancia = algoritmo_genetico(problem, N=500, mutacion=0.15, elitismo=0.4, generaciones=250)
print(f"Mejor solución encontrada: {sol} con una distancia de {distancia}")

Generación 1: Mejor distancia = 3944
Generación 2: Mejor distancia = 3814
Generación 3: Mejor distancia = 3568
Generación 4: Mejor distancia = 3568
Generación 5: Mejor distancia = 3507
Generación 6: Mejor distancia = 3507
Generación 7: Mejor distancia = 3430
Generación 8: Mejor distancia = 3333
Generación 9: Mejor distancia = 3280
Generación 10: Mejor distancia = 3188
Generación 11: Mejor distancia = 3188
Generación 12: Mejor distancia = 3124
Generación 13: Mejor distancia = 3115
Generación 14: Mejor distancia = 3089
Generación 15: Mejor distancia = 2965
Generación 16: Mejor distancia = 2965
Generación 17: Mejor distancia = 2876
Generación 18: Mejor distancia = 2876
Generación 19: Mejor distancia = 2876
Generación 20: Mejor distancia = 2839
Generación 21: Mejor distancia = 2839
Generación 22: Mejor distancia = 2807
Generación 23: Mejor distancia = 2793
Generación 24: Mejor distancia = 2793
Generación 25: Mejor distancia = 2793
Generación 26: Mejor distancia = 2744
Generación 27: Mejor 